# GPT-5-Mini GMS API 테스트

이 노트북은 `main.py`의 프롬프트 로직을 `gpt-5-mini` 모델과 GMS API를 통해 테스트하기 위해 작성되었습니다.

In [1]:
import requests
import json
import datetime
import os

# GMS API 키 설정 (환경 변수에서 가져오거나 직접 입력)
os.environ["GMS_KEY"] = "S14P02AQ06-88cb9f96-1505-4bb8-b068-57fe14afebee" # 필요한 경우 주석 해제 후 입력
GMS_KEY = os.getenv("GMS_KEY")

if not GMS_KEY:
    print("⚠️ 경고: GMS_KEY 환경 변수가 설정되지 않았습니다. API 호출이 실패할 수 있습니다.")

API_URL = "https://gms.ssafy.io/gmsapi/api.openai.com/v1/chat/completions"

In [ ]:
# --- Mock Data 설정 (main.py의 실행 문맥 시뮬레이션) ---

class MockRequest:
    def __init__(self, name, dream_content, gender, birth_date):
        self.name = name
        self.dream_content = dream_content
        self.gender = gender
        self.birth_date = birth_date

class MockFortune:
    def __init__(self, summary):
        self.summary = summary

# 테스트용 데이터 입력
request = MockRequest(
    name="김현빈",
    dream_content="넓은 초원에서 유니콘을 타고 하늘을 나는 꿈을 꾸었습니다. 바람이 시원하게 느껴졌고 기분이 매우 상쾌했습니다.",
    gender="male",
    birth_date="2000-05-05"
)

dream_interpretation = "이 꿈은 자유와 높은 이상을 추구하는 심리를 나타냅니다. 유니콘은 행운과 순수함을 상징하며, 하늘을 나는 것은 현재의 제약에서 벗어나 비상하고 싶은 욕구를 반영합니다. 긍정적인 변화가 찾아올 징조입니다."

naver_fortune = MockFortune(
    summary="오늘은 예상치 못한 귀인을 만나 도움을 받을 수 있는 날입니다. 새로운 시도를 두려워하지 마세요."
)

In [7]:
# --- main.py (153-205) 로직 적용 ---

today_date = datetime.datetime.now().strftime("%Y-%m-%d")

color_options = [
    {"name": "회색"},
    {"name": "갈색"},
    {"name": "주황색"},
    {"name": "노란색"},
    {"name": "초록색"},
    {"name": "파란색"},
    {"name": "보라색"},
    {"name": "분홍색"},
    {"name": "빨간색"}
]

system_prompt = (
    "당신은 오늘 하루의 운세를 알려주는 주술가입니다. "
    "사용자의 꿈과 운세를 종합 분석하여 인과관계가 있는 통찰을 제공하세요. "
    "반드시 주어진 JSON 형식으로만 응답해야 합니다."
)

user_prompt = f"""
### 사용자 정보
- 이름: {request.name}
- 성별: {request.gender}
- 생년월일: {request.birth_date}
- 날짜: {today_date}

### 입력 데이터
1. **꿈 내용:**
{request.dream_content.strip()}

2. **꿈 해몽 분석:**
{dream_interpretation.strip()}

3. **오늘의 운세:**
{naver_fortune.summary.strip()}

4. **선택 가능한 행운의 색상 목록:**
{json.dumps(color_options, ensure_ascii=False)}

### 요청 사항
위 정보를 종합하여 JSON 형식으로 운세 리포트를 작성해주세요.
특히 생년월일과 성별을 고려하여 오늘 사용자에게 행운을 가져다줄 아이템을 추천해주세요.

### 출력 형식 (JSON Only)
{{
  "title": "{request.name}님의 {today_date} 통합 운세",
  "today_fortune_summary": "운세 내용 작성...",
  "action_guide": "행동 지침 작성...",
  "lucky_color": {{
      "name": "색상명 (목록에 있는 것만)",
      "reason": "선정 이유 작성"
  }},
  "lucky_item": {{
      "name": "추천 아이템 명",
      "reason": "생년월일/성별 기반 추천 이유"
  }}
}}
"""

print("--- User Prompt 미리보기 ---")
print(user_prompt)

--- User Prompt 미리보기 ---

### 사용자 정보
- 이름: 김싸피
- 성별: male
- 생년월일: 1998-05-20
- 날짜: 2025-12-02

### 입력 데이터
1. **꿈 내용:**
넓은 초원에서 유니콘을 타고 하늘을 나는 꿈을 꾸었습니다. 바람이 시원하게 느껴졌고 기분이 매우 상쾌했습니다.

2. **꿈 해몽 분석:**
이 꿈은 자유와 높은 이상을 추구하는 심리를 나타냅니다. 유니콘은 행운과 순수함을 상징하며, 하늘을 나는 것은 현재의 제약에서 벗어나 비상하고 싶은 욕구를 반영합니다. 긍정적인 변화가 찾아올 징조입니다.

3. **오늘의 운세:**
오늘은 예상치 못한 귀인을 만나 도움을 받을 수 있는 날입니다. 새로운 시도를 두려워하지 마세요.

4. **선택 가능한 행운의 색상 목록:**
[{"name": "회색"}, {"name": "갈색"}, {"name": "주황색"}, {"name": "노란색"}, {"name": "초록색"}, {"name": "파란색"}, {"name": "보라색"}, {"name": "분홍색"}, {"name": "빨간색"}]

### 요청 사항
위 정보를 종합하여 JSON 형식으로 운세 리포트를 작성해주세요.
특히 생년월일과 성별을 고려하여 오늘 사용자에게 행운을 가져다줄 아이템을 추천해주세요.

### 출력 형식 (JSON Only)
{
  "title": "김싸피님의 2025-12-02 통합 운세",
  "today_fortune_summary": "운세 내용 작성...",
  "action_guide": "행동 지침 작성...",
  "lucky_color": {
      "name": "색상명 (목록에 있는 것만)",
      "reason": "선정 이유 작성"
  },
  "lucky_item": {
      "name": "추천 아이템 명",
      "reason": "생년월일/성별 기반 추천 이유"
  }
}



In [8]:
# --- API 요청 전송 ---

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {GMS_KEY}"
}

payload = {
    "model": "gpt-5-mini",
    "messages": [
        { 
            "role": "developer", 
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]
}

print(f"\n📡 API 요청 중... ({API_URL})")

try:
    response = requests.post(API_URL, headers=headers, json=payload)
    response.raise_for_status()
    
    result = response.json()
    
    # 결과 출력
    if 'choices' in result and len(result['choices']) > 0:
        content = result['choices'][0]['message']['content']
        print("\n✅ 응답 수신 성공:")
        print(content)
        
        # JSON 파싱 테스트
        try:
            # 마크다운 코드 블록(```json ... ```) 제거 처리
            clean_content = content.replace('```json', '').replace('```', '').strip()
            parsed_json = json.loads(clean_content)
            print("\n✨ JSON 파싱 성공!")
            print(json.dumps(parsed_json, indent=2, ensure_ascii=False))
        except json.JSONDecodeError:
            print("\n⚠️ JSON 파싱 실패 (순수 JSON 형식이 아닐 수 있습니다)")
    else:
        print("\n⚠️ 응답에 내용이 없습니다.")
        print(result)
        
except requests.exceptions.RequestException as e:
    print(f"\n❌ API 요청 에러: {e}")
    if 'response' in locals():
        print(f"응답 내용: {response.text}")


📡 API 요청 중... (https://gms.ssafy.io/gmsapi/api.openai.com/v1/chat/completions)

✅ 응답 수신 성공:
{
  "title": "김싸피님의 2025-12-02 통합 운세",
  "today_fortune_summary": "넓은 초원에서 유니콘을 타고 하늘을 나는 꿈은 자유와 높은 이상, 그리고 순수한 행운의 도래를 알립니다. 유니콘은 복과 순수함을, 비상하는 이미지는 현재의 제약에서 벗어나려는 강한 욕구와 그것이 현실로 연결될 징후를 의미합니다. 오늘은 예상치 못한 귀인이 나타나 도움을 주거나 새로운 기회가 열릴 가능성이 큽니다. 다만 변화가 갑작스럽게 오기 쉬우니 마음을 열되 현실적인 준비를 병행하면 이득을 더 크게 볼 수 있습니다.",
  "action_guide": "새로운 제안이나 만남에 열린 태도를 유지하세요. 즉흥적 결정은 좋지만 작은 준비(연락처 정리, 간단한 자기소개 준비 등)를 해두면 기회를 안정적으로 잡을 수 있습니다. 오늘 만나는 사람의 말에 경청하고 질문을 통해 진의를 파악하세요. 바깥 활동이나 짧은 산책으로 마음을 가다듬으면 직관이 살아납니다. 파란색 계열의 소지품을 가까이 두고(휴대품, 액세서리 등) 귀인과의 소통에 신경 쓰십시오.",
  "lucky_color": {
    "name": "파란색",
    "reason": "꿈에서 느낀 시원한 바람과 하늘의 이미지에 가장 잘 맞는 색입니다. 파란색은 평온과 신뢰를 증진시켜 새로운 만남에서 긍정적 인상을 주고, 귀인과의 소통을 원활하게 해 오늘의 기회를 잘 연결시켜 줍니다."
  },
  "lucky_item": {
    "name": "은색 반지(작은 파란 보석 장식)",
    "reason": "1998-05-20 출생(5월 20일)은 현실적인 성향(황소자리 기운 성향)과 변화에 대한 욕구가 공존합니다. 남성으로서 은색 액세서리는 차분하고 신뢰감 있는 이미지를 주어 타인의 도움을 끌어들이기 쉽습니다. 여기에 파란